In [87]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../../src/')
from train import load_data
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor
import magic
import torch
wandb.login()
api = wandb.Api()

In [88]:
from procrustes import Procrustes

In [89]:
from plotly3d.plot import scatter

In [90]:
entity = "xingzhis"
project = "dmae"
sweep_id = 'mfsp6l3q'
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
# Initialize an empty list to store run data
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

In [91]:
df['data.name']

0            hemisphere
1            hemisphere
2      make_mix_surface
3    hemisphere_hi_dim2
4    hemisphere_hi_dim2
5      make_mix_surface
6         eb_subset_all
7         eb_subset_all
Name: data.name, dtype: object

In [92]:
df.columns

Index(['train/loss_epoch', 'validation/loss', '_timestamp',
       'train/reconstr_loss_step', 'validation/reconstr_loss',
       'train/reconstr_loss_epoch', 'epoch', '_runtime',
       'train/dist_loss_epoch', '_wandb', 'train/loss_step',
       'trainer/global_step', 'train/dist_loss_step', 'validation/dist_loss',
       '_step', 'data', 'loss', 'path', 'logger', 'decoder', 'encoder',
       'training', 'data.name', 'data.root', 'dimensions', 'preprocessor',
       'cfg/data/name', 'cfg/data/root', 'cfg/path/root', 'training.mode',
       'cfg/path/model', 'cfg/logger/tags', 'cfg/training/lr',
       'decoder.dropout', 'encoder.dropout', 'cfg/data/filetype',
       'cfg/logger/entity', 'cfg/training/mode', 'cfg/training/seed',
       'dimensions.latent', 'loss.weights.dist', 'cfg/logger/project',
       'decoder.batch_norm', 'encoder.batch_norm', 'loss.weights.cycle',
       'cfg/decoder/dropout', 'cfg/dimensions/data', 'cfg/encoder/dropout',
       'loss.dist_mse_decay', 'training.

In [93]:
df['data.name'].value_counts()

data.name
hemisphere            2
make_mix_surface      2
hemisphere_hi_dim2    2
eb_subset_all         2
Name: count, dtype: int64

In [94]:
df[(df['data.name'] == 'hemisphere_hi_dim2') & (df['training.mode'] == 'end2end')].sort_values(['validation/dist_loss', 'validation/reconstr_loss'])[['validation/loss', 'validation/dist_loss', 'validation/reconstr_loss', 'run_id', 'cfg/encoder/dropout', 'cfg/loss/dist_mse_decay', 'cfg/loss/weights/cycle', 'cfg/dimensions/latent']]

,validation/loss,validation/dist_loss,validation/reconstr_loss,run_id,cfg/encoder/dropout,cfg/loss/dist_mse_decay,cfg/loss/weights/cycle,cfg/dimensions/latent
4,106.180916,1.085196,69.333214,qn6eqnim,0,0.5,0,3
3,108.764595,1.091488,75.886055,ks5edwov,0,0.5,0,2


In [95]:
df[(df['data.name'] == 'hemisphere_hi_dim2') & (df['training.mode'] == 'end2end') & (df['cfg/dimensions/latent'] == 3)].sort_values(['validation/dist_loss', 'validation/reconstr_loss'])

,train/loss_epoch,validation/loss,_timestamp,train/reconstr_loss_step,validation/reconstr_loss,train/reconstr_loss_epoch,epoch,_runtime,train/dist_loss_epoch,_wandb,...,loss.weights.cycle_dist,cfg/decoder/layer_widths,cfg/encoder/layer_widths,cfg/training/accelerator,cfg/loss/weights/reconstr,cfg/training/weight_decay,cfg/loss/weights/cycle_dist,cfg/training/log_every_n_steps,cfg/training/train_valid_split,run_id
4,15.533262,106.180916,1.713748e+09,46.562935,69.333214,46.920654,49,4262.757383,0.006701,{'runtime': 4262},...,0,"[64, 128, 256]","[256, 128, 64]",cuda,0.32,0.0001,0,100,0.8,qn6eqnim


In [96]:
# run_id = 'ukca8aig'
run_id = 'qn6eqnim'
run = api.run(f"{entity}/{project}/{run_id}")
folder_path = '../../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
# data_path2 = os.path.join(cfg., cfg.data.name + cfg.data.filetype)
# data = np.load(data_path2, allow_pickle=True)
# x_tensor = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
# x_tensor_normalized = model.normalize(x_tensor)
cfg.data.root = '../' + cfg.data.root
# trainloader, valloader, X, phate_coords, colors, preprocessor = load_data(cfg)
model = Autoencoder.load_from_checkpoint(ckpt_path)
# cfg.dimensions.data = X.shape[1]
# cfg.training.mode = 'end2end'
# model = Autoencoder(cfg, preprocessor)
# model = Autoencoder.load_from_checkpoint(ckpt_path, preprocessor=preprocessor)
data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    model.eval()
    x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
    z = model.encoder(x)
    xh = model.decoder(z)
zc = z.cpu().numpy()
scatter(zc, data['mask_x'], s=2, alpha=0.2)


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning:

Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.



In [111]:
scatter(zc, data['mask_x'], s=2, alpha=0.2)

In [113]:
df[(df['data.name'] == 'eb_subset_all') & (df['training.mode'] == 'end2end') & (df['cfg/dimensions/latent'] == 3)].sort_values(['validation/dist_loss', 'validation/reconstr_loss'])

,train/loss_epoch,validation/loss,_timestamp,train/reconstr_loss_step,validation/reconstr_loss,train/reconstr_loss_epoch,epoch,_runtime,train/dist_loss_epoch,_wandb,...,loss.weights.cycle_dist,cfg/decoder/layer_widths,cfg/encoder/layer_widths,cfg/training/accelerator,cfg/loss/weights/reconstr,cfg/training/weight_decay,cfg/loss/weights/cycle_dist,cfg/training/log_every_n_steps,cfg/training/train_valid_split,run_id
6,7.718573,80.003349,1.713748e+09,22.900043,48.810444,22.044889,49,4353.550819,0.008582,{'runtime': 4353},...,0,"[64, 128, 256]","[256, 128, 64]",cuda,0.32,0.0001,0,100,0.8,r2a11lv3


In [114]:
# run_id = 'ukca8aig'
run_id = 'r2a11lv3'
run = api.run(f"{entity}/{project}/{run_id}")
folder_path = '../../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
# data_path2 = os.path.join(cfg., cfg.data.name + cfg.data.filetype)
# data = np.load(data_path2, allow_pickle=True)
# x_tensor = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
# x_tensor_normalized = model.normalize(x_tensor)
cfg.data.root = '../' + cfg.data.root
# trainloader, valloader, X, phate_coords, colors, preprocessor = load_data(cfg)
model = Autoencoder.load_from_checkpoint(ckpt_path)
# cfg.dimensions.data = X.shape[1]
# cfg.training.mode = 'end2end'
# model = Autoencoder(cfg, preprocessor)
# model = Autoencoder.load_from_checkpoint(ckpt_path, preprocessor=preprocessor)
data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    model.eval()
    x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
    z = model.encoder(x)
    xh = model.decoder(z)
zc = z.cpu().numpy()
scatter(zc, data['mask_x'], s=2, alpha=0.2)


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning:

Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.

